# 3-9 시계열 데이터의 처리

In [2]:
import os
import pandas as pd

base_url = 'https://raw.githubusercontent.com/practical-jupyter/sample-data/master/anime/'
anime_stock_price_csv = os.path.join(base_url, 'anime_stock_price.csv')

df = pd.read_csv(anime_stock_price_csv, index_col=0, parse_dates=['Date'])
df.head()

,TOEI ANIMATION,IG Port
Date,,
2015-01-01,3356.86,1201.51
2015-01-02,3356.86,1201.51
2015-01-05,3396.12,1218.44
2015-01-06,3361.77,1201.51
2015-01-07,3297.97,1202.51


### 시계열 데이터로 사용된 함수
- pct_change() 메서드를 사용하여 하나 전의 값으로부터의 변화율을 산출한다.

In [3]:
pd.options.display.max_rows = 10 # pandas로 실행하는 행 수를 지정

pct_change = df['TOEI ANIMATION'].pct_change()
pct_change

Date
2015-01-01         NaN
2015-01-02    0.000000
2015-01-05    0.011695
2015-01-06   -0.010114
2015-01-07   -0.018978
                ...   
2016-12-26    0.001725
2016-12-27    0.010326
2016-12-28    0.015333
2016-12-29    0.013422
2016-12-30    0.000000
Name: TOEI ANIMATION, Length: 522, dtype: float64

cum_prod() 메서드를 사용하여 누적값을 산출할 수 있다.

In [15]:
cumulative_returns = (pct_change + 1).cumprod()
cumulative_returns[0] = 1
cumulative_returns

Date
2015-01-01    1.000000
2015-01-02    1.000000
2015-01-05    1.011695
2015-01-06    1.001463
2015-01-07    0.982457
                ...   
2016-12-26    1.722833
2016-12-27    1.740624
2016-12-28    1.767312
2016-12-29    1.791034
2016-12-30    1.791034
Name: TOEI ANIMATION, Length: 522, dtype: float64

rolling() 메서드를 사용하여 데이터의 범위를 이동시키면서 함수를 적용할 수 있다. 

In [17]:
df['TOEI ANIMATION'].rolling(5).mean()

Date
2015-01-01         NaN
2015-01-02         NaN
2015-01-05         NaN
2015-01-06         NaN
2015-01-07    3353.916
                ...   
2016-12-26    5793.260
2016-12-27    5799.232
2016-12-28    5821.132
2016-12-29    5868.912
2016-12-30    5916.692
Name: TOEI ANIMATION, Length: 522, dtype: float64

위 예제는 mean() 메서드를 사용하고 있지만 임의의 함수를 적용할 수 있다.

In [20]:
import numpy as np

def historical_volatility(x):
    logreturns = np.diff(np.log(x)) # 로그 수익률   np.diff 함수는 해당 array의 직전수와의 차이를 array로 돌려줌.
    return np.sqrt(365 * logreturns.var())  # log수익의 차이들의 분산을 365로 곱하고 루트를 씌움. 즉 log 표준편차 정도될듯.

df['TOEI ANIMATION'].rolling(20).apply(historical_volatility)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


Date
2015-01-01         NaN
2015-01-02         NaN
2015-01-05         NaN
2015-01-06         NaN
2015-01-07         NaN
                ...   
2016-12-26    0.158355
2016-12-27    0.163235
2016-12-28    0.170891
2016-12-29    0.172681
2016-12-30    0.151983
Name: TOEI ANIMATION, Length: 522, dtype: float64

## DatetimeIndex
- DatetimeIndexsms datetime 형에 특화된 처리가 가능한 Index이다. 
- pandas.date_range() 함수는 지정한 주기의 DatetimeIndex를 작성한다.
- start, end, periods, freq, tz(타임존), normalize(시각부분을 둥글게한다??), name(문자열로 지정한다.), 
- closed(지정한 쪽의 기간을 닫는다.left하면 마지막일시가 제외, right하면 최초일시가 제외.

In [23]:
ix = pd.date_range('2017-01','2017-02',freq='1H')
ix

DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 01:00:00',
               '2017-01-01 02:00:00', '2017-01-01 03:00:00',
               '2017-01-01 04:00:00', '2017-01-01 05:00:00',
               '2017-01-01 06:00:00', '2017-01-01 07:00:00',
               '2017-01-01 08:00:00', '2017-01-01 09:00:00',
               ...
               '2017-01-31 15:00:00', '2017-01-31 16:00:00',
               '2017-01-31 17:00:00', '2017-01-31 18:00:00',
               '2017-01-31 19:00:00', '2017-01-31 20:00:00',
               '2017-01-31 21:00:00', '2017-01-31 22:00:00',
               '2017-01-31 23:00:00', '2017-02-01 00:00:00'],
              dtype='datetime64[ns]', length=745, freq='H')

Series의 index로 사용하는 경우

In [25]:
time_series = pd.Series(np.arange(len(ix)), index=ix)
time_series

2017-01-01 00:00:00      0
2017-01-01 01:00:00      1
2017-01-01 02:00:00      2
2017-01-01 03:00:00      3
2017-01-01 04:00:00      4
                      ... 
2017-01-31 20:00:00    740
2017-01-31 21:00:00    741
2017-01-31 22:00:00    742
2017-01-31 23:00:00    743
2017-02-01 00:00:00    744
Freq: H, Length: 745, dtype: int32

### 시계열 데이터를 추출하기
- DatetimeIndex의 인덱서에는 datetime혀오가 문자열형 양쪽을 지정할 수 있다. 

In [26]:
from datetime import datetime

# 인덱서에 datetime형 값 지정
df.loc[datetime(2016,1,4)]

TOEI ANIMATION    5699.74
IG Port            822.66
Name: 2016-01-04 00:00:00, dtype: float64

In [28]:
#인덱서에 날짜형식의 문자열을 지정1
df.loc['2016-01-04']

TOEI ANIMATION    5699.74
IG Port            822.66
Name: 2016-01-04 00:00:00, dtype: float64

In [29]:
# 인덱서에 날짜형식의 문자열을 지정2
df.loc['Jan-04-2016']

TOEI ANIMATION    5699.74
IG Port            822.66
Name: 2016-01-04 00:00:00, dtype: float64

특정 년도나 월의 데이터만 추출하는 경우

In [31]:
print(df.loc['2015'].head())

            TOEI ANIMATION  IG Port
Date                               
2015-01-01         3356.86  1201.51
2015-01-02         3356.86  1201.51
2015-01-05         3396.12  1218.44
2015-01-06         3361.77  1201.51
2015-01-07         3297.97  1202.51


In [32]:
print(df.loc['2015-05'].head())

            TOEI ANIMATION  IG Port
Date                               
2015-05-01         3567.29  1168.66
2015-05-04         3567.29  1168.66
2015-05-05         3567.29  1168.66
2015-05-06         3567.29  1168.66
2015-05-07         3577.18  1169.66


In [33]:
# 년월등을 슬라이스하는 경우
print(df.loc['2015-12':'2016-01'])

            TOEI ANIMATION  IG Port
Date                               
2015-12-01         5947.13   910.41
2015-12-02         5917.44   896.45
2015-12-03         5917.44   893.46
2015-12-04         5867.97   888.47
2015-12-07         5917.44   892.46
...                    ...      ...
2016-01-25         5452.36   704.99
2016-01-26         5491.94   702.00
2016-01-27         5521.63   721.94
2016-01-28         5679.95   733.91
2016-01-29         5670.06   739.89

[44 rows x 2 columns]


In [34]:
# 지정한 시각만의 데이터를 추출하는 경우
from datetime import time

time_series.loc[time(9,0)]

2017-01-01 09:00:00      9
2017-01-02 09:00:00     33
2017-01-03 09:00:00     57
2017-01-04 09:00:00     81
2017-01-05 09:00:00    105
                      ... 
2017-01-27 09:00:00    633
2017-01-28 09:00:00    657
2017-01-29 09:00:00    681
2017-01-30 09:00:00    705
2017-01-31 09:00:00    729
Freq: 24H, Length: 31, dtype: int32

In [35]:
# 지정한 시간대만 추출하는 경우
# between_time() 메서드를 이용하여 지정한 시간대만 추출할 수 있다.
time_series.between_time(time(9,0),time(12,0))

2017-01-01 09:00:00      9
2017-01-01 10:00:00     10
2017-01-01 11:00:00     11
2017-01-01 12:00:00     12
2017-01-02 09:00:00     33
                      ... 
2017-01-30 12:00:00    708
2017-01-31 09:00:00    729
2017-01-31 10:00:00    730
2017-01-31 11:00:00    731
2017-01-31 12:00:00    732
Length: 124, dtype: int32

### 리샘플링
- resample() 메서드를 이용하여 시계열 데이터의 빈도를 변환할 수 있다.
- 일별 데이터를 주별이나 월별 등의 데이터로 변환할 수 있다.

In [37]:
df['TOEI ANIMATION'].resample('M').mean().head()

Date
2015-01-31    3647.080000
2015-02-28    3612.302500
2015-03-31    3625.770455
2015-04-30    3477.555455
2015-05-31    3653.990476
Freq: M, Name: TOEI ANIMATION, dtype: float64

In [41]:
df['TOEI ANIMATION'].resample('M').count()

Date
2015-01-31    22
2015-02-28    20
2015-03-31    22
2015-04-30    22
2015-05-31    21
              ..
2016-08-31    23
2016-09-30    22
2016-10-31    21
2016-11-30    22
2016-12-31    22
Freq: M, Name: TOEI ANIMATION, Length: 24, dtype: int64

In [49]:
#  일별 마감가를 주봉으로 변환
df['TOEI ANIMATION'].resample('W').ohlc()

,open,high,low,close
Date,,,,
2015-01-04,3356.86,3356.86,3356.86,3356.86
2015-01-11,3396.12,3513.90,3297.97,3513.90
2015-01-18,3513.90,3872.16,3435.38,3872.16
2015-01-25,3877.07,3877.07,3739.66,3739.66
2015-02-01,3774.01,3965.41,3774.01,3965.41
...,...,...,...,...
2016-12-04,5803.22,5872.89,5803.22,5872.89
2016-12-11,5773.35,5862.94,5773.35,5773.35
2016-12-18,5713.63,5783.31,5713.63,5783.31


In [50]:
# https://stackoverflow.com/questions/36222928/pandas-ohlc-aggregation-on-ohlc-data
# 위는 이미 ohlc로 되어있는 데이터를 다른 단위로 바꿔서 하고 싶을 때임. 
# 답변에는 resmaple.agg(dict) 방식으로 해결했으니 참고.